In [1]:
# !pip install -U torch transformers datasets accelerate sentencepiece

In [2]:
# !pip install -U datasets huggingface_hub

### 데이터셋 다운로드

https://github.com/songys/Chatbot_data

프로젝트 폴더에 ChatbotData.csv 파일을 두세요. (형식: Q,A,label)

In [3]:
import os
import csv
import random
from typing import List, Dict

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer, TrainingArguments, pipeline, set_seed
)

In [4]:
# 0) Device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [5]:
# 환경 설정

MODEL_NAME      = "skt/kogpt2-base-v2"
DATA_PATH       = "./ChatbotData.csv"   # 로컬 CSV 경로 (Q,A,label)
OUTPUT_DIR      = "./kogpt2-chatbot"
MAX_LENGTH      = 128
EPOCHS          = 10          # 데모용 2번 권장/ 여유되면 10 권장
LR              = 5e-5
TRAIN_RATIO     = 0.98       # 98% train / 2% valid (데이터가 작은 편이라 검증을 아주 작게)
BATCH_PER_DEV   = 8
GRAD_ACC_STEPS  = 2
SEED            = 42

In [6]:
# 1) 로컬 CSV 읽기

def load_chatbot_csv(path: str) -> List[Dict[str, str]]:
    """
    Chatbot_data.csv (Q,A,label) 형식을 읽어 {"Q": "...", "A": "..."} 리스트로 반환
    """
    samples = []
    with open(path, encoding="utf-8") as f:
        reader = csv.DictReader(f)
        # 파일에 header가 없으면 DictReader가 오동작합니다.
        # header가 정확히 Q,A,label 인지 확인하세요.
        # (header가 다르다면 아래 키 이름을 수정)
        for row in reader:
            q = (row.get("Q") or "").strip()
            a = (row.get("A") or "").strip()
            if q and a:
                samples.append({"Q": q, "A": a})
    return samples

In [7]:
samples = load_chatbot_csv(DATA_PATH)
print(f"샘플 개수: {len(samples)}")
if len(samples) == 0:
    # 예비 안전장치: CSV가 없거나 비어 있을 때, 아주 작은 더미 데이터로라도 실행 가능
    print("경고: CSV 샘플이 0개입니다. 더미 데이터로 대체합니다.")
    samples = [
        {"Q": "안녕", "A": "안녕하세요. 무엇을 도와드릴까요?"},
        {"Q": "날씨 어때?", "A": "지역에 따라 다르지만, 오늘은 맑을 확률이 높아요."},
        {"Q": "고마워", "A": "별말씀을요!"}
    ]

샘플 개수: 11823


In [8]:
# 2) 프롬프트 포맷 정의

# KoGPT2는 일반 Causal LM입니다. 간단한 포맷으로 지도학습합니다.
# 예: "사용자: ...\n봇: ...</eos>"
USER_TAG = "사용자"
BOT_TAG  = "봇"

def make_prompt(q: str, a: str) -> str:
    return f"{USER_TAG}: {q}\n{BOT_TAG}: {a}"


In [9]:
# 학습 텍스트 만들기
texts = [make_prompt(ex["Q"], ex["A"]) for ex in samples]

In [10]:
# 3) 데이터 분할 (train/valid)

n_total = len(texts)
n_train = max(1, int(n_total * TRAIN_RATIO))
indices = list(range(n_total))
random.shuffle(indices)

train_idx = indices[:n_train]
valid_idx = indices[n_train:] if n_total > 1 else []

train_texts = [texts[i] for i in train_idx]
valid_texts = [texts[i] for i in valid_idx] if valid_idx else [texts[-1]]

raw = DatasetDict({
    "train": Dataset.from_dict({"text": train_texts}),
    "validation": Dataset.from_dict({"text": valid_texts})
})
print(raw)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 11586
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 237
    })
})


In [11]:
# 4) 토크나이저/모델

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# KoGPT2는 pad_token 설정이 필요합니다. eos를 pad로 사용
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
model.resize_token_embeddings(len(tokenizer))
model.to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51201, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51201, bias=False)
)

In [12]:
# 5) 토크나이즈

def tokenize_fn(batch):
    # KoGPT2용: causal LM => 입력=라벨
    # padding="max_length"는 작은 데이터에서 안정적으로 돌아가게 해줍니다.
    return tokenizer(
        batch["text"],
        max_length=MAX_LENGTH,
        truncation=True,
        padding="max_length",
        return_attention_mask=True
    )

In [13]:
tokenized = raw.map(tokenize_fn, batched=True, remove_columns=raw["train"].column_names)

Map:   0%|          | 0/11586 [00:00<?, ? examples/s]

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

In [14]:
# 6) Data Collator

# labels는 input_ids와 동일 (causal LM)
collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [15]:
# 7) 학습 설정

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_PER_DEV,
    per_device_eval_batch_size=BATCH_PER_DEV,
    gradient_accumulation_steps=GRAD_ACC_STEPS,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    warmup_steps=100,
    eval_strategy="steps",
    eval_steps=200,
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

In [16]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    data_collator=collator
)

In [17]:
# 8) 학습

trainer.train()
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
200,2.983900,2.812635
400,2.861500,2.696678
600,2.733500,2.596201
800,2.341700,2.555506
1000,2.257700,2.541821
1200,2.243000,2.464952
1400,2.204900,2.422349
1600,1.774100,2.418139
1800,1.806500,2.435328
2000,1.810400,2.382254


('./kogpt2-chatbot/tokenizer_config.json',
 './kogpt2-chatbot/special_tokens_map.json',
 './kogpt2-chatbot/vocab.json',
 './kogpt2-chatbot/merges.txt',
 './kogpt2-chatbot/added_tokens.json',
 './kogpt2-chatbot/tokenizer.json')

In [18]:
# 9) 추론 파이프라인

gen = pipeline(
    "text-generation",
    model=OUTPUT_DIR,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

Device set to use cuda:0


In [19]:
def chat_once(user_text: str,
              max_new_tokens: int = 64,
              temperature: float = 0.8,
              top_p: float = 0.9) -> str:
    prompt = f"{USER_TAG}: {user_text}\n{BOT_TAG}:"
    out = gen(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id
    )[0]["generated_text"]
    # "봇:" 이후만 추출
    reply = out.split(f"{BOT_TAG}:")[-1]
    # 다음 사용자 태그가 있으면 그 전까지만
    if f"\n{USER_TAG}:" in reply:
        reply = reply.split(f"\n{USER_TAG}:")[0]
    return reply.strip()

In [22]:
# 10) 간단한 테스트

tests = ["빨리 들어가셔요!", "저녁 뭐 먹을까요?", "요즘 스트레스가 많아요"]
for t in tests:
    print(f"\n[사용자] {t}")
    print(f"[봇   ] {chat_once(t)}")


[사용자] 빨리 들어가셔요!
[봇   ] 오늘부터 하세요. 제가 위로해 드릴게요. 저한테 말해보세요. 곧 다가올 거예요. 그럼요. 제가 위로해 드릴게요. 제가 위로해 드릴게요. 제가 위로해 드리고 싶어요. 맘이 놓아 주길 바랄게요. 아직은 힘들어도 괜찮아요. 더

[사용자] 저녁 뭐 먹을까요?
[봇   ] 맛있는 거 드세요. 든든하게 드세요. 기분 전환이 되길 바랄게요. 맛있는 거 드세요. 맛있는 거 드세요!!!! 저에게 기대세요. 항상 함께 했던 시간들이었죠. 잘 지내요. 힘들었을텐데. 다시 그리움을 헤아리지 마세요.

[사용자] 요즘 스트레스가 많아요
[봇   ] 좋은 소식이 들리길 바랄게요. 좋은 소식이 들리길 바랄게요. 다른 사람 마음에게 기대보는 건 어
